# Onboarding Runbook

## Goals

- Understand The Onboarding Process
- Understand How To Set Up A Global Namespace (GNS)
- Understand How A GNS Allows Distributed Service Meshes

## Login

Setup your kubeconfig for cluster 1 (named acme-c1) and cluster 2 (named acme-c2) as two kubeconfig files for each cluster.

In [ ]:
export KUBECONFIG=./kubeconfig-acme-c1.yml:./kubeconfig-acme-c2.yml

In [ ]:
kubectl config get-contexts

Login into your [Tanzu Service Mesh tenant](./imgs/1-homepage.png).

## [Onboard Cluster 1](./imgs/2-onboard-a-cluster.png)

In [ ]:
kubectl config use-context acme-c1

Enter a [Unique Cluster Name](./imgs/3-onboard-a-cluster-unique-name.png)

Paste [Registration Script 1](./imgs/4-onboard-a-cluster-register.png) Below

Paste Registration Script 2 Below

### Gotcha: Slow Or Incomplete Verification

The namespace “allspark” is created with 3 pods (allspark-ws-proxy, ecr-read-only--renew-token and k8s-cluster-manager). If any pod is in a state of ImagePullBackOff, just delete the pod to make the process faster.

In [ ]:
kubectl -n allspark get pods

### Gotcha: TMC + AWS

Clicking on the Install Service Mesh button will install the component install in the “istio-system” namespace. Out of the box TMC on AWS fails this step. The error is “CreateContainerConfigError” for the “istio-init-crd” pod. The following cluster role binding is required.

In [ ]:
kubectl create clusterrolebinding privileged-role-binding \
--clusterrole=vmware-system-tmc-psp-privileged \
--group=system:authenticated

[Install TSM](./imgs/5-onboard-a-cluster-tsm-install.png)

[Installing Dependencies](./imgs/6-onboard-a-cluster-tsm-install-deps.png)

In [ ]:
kubectl -n istio-system get pods

[Completed](./imgs/7-onboard-a-cluster-tsm-install-complete.png)

In [ ]:
k -n istio-system get svc istio-ingressgateway

### Onboard Cluster 2

Repeat the previous process using acme-c2 as the unique name.

In [ ]:
kubectl config use-context acme-c2

Paste Registration Script 1 Below

Paste Registration Script 2 Below

In [ ]:
kubectl -n allspark get pods

In [ ]:
kubectl -n istio-system get pods

[Clusters Onboarded](./imgs/8-clusters-onboarded.png)

## Deploy Acme Fitness On Cluster 1

![Acme Fitness](./imgs/9-acmeshop.png)

Clone from Acme Fitness Demo

In [ ]:
git clone -b dkalani-dev3 https://github.com/vmwarecloudadvocacy/acme_fitness_demo.git

In [ ]:
kubectl config use-context acme-c1

In [ ]:
kubectl label ns default istio-injection=enabled

In [ ]:
kubectl apply -f acme_fitness_demo/kubernetes-manifests/secrets.yaml

In [ ]:
kubectl apply -f acme_fitness_demo/kubernetes-manifests/acme_fitness_cluster1.yaml

In [ ]:
kubectl apply -f acme_fitness_demo/kubernetes-manifests/acme_fitness_cluster2.yaml

In [ ]:
kubectl set env deploy shopping CATALOG_HOST=catalog

In [ ]:
kubectl get all

In [ ]:
kubectl apply -f acme_fitness_demo/istio-manifests/gateway.yaml

In [ ]:
kubectl -n istio-system get svc istio-ingressgateway

In [ ]:
kubectl apply -f acme_fitness_demo/traffic-generator/loadgen.yaml

[Acme Cluster 1 fabric overview](./imgs/10-acme-c1-fabric-overview.png)

Open browser to URL

## Duplicate Catalog Service On Cluster 2

In [ ]:
kubectl config use-context acme-c2

In [ ]:
kubectl label ns default istio-injection=enabled

In [ ]:
kubectl apply -f acme_fitness_demo/kubernetes-manifests/secrets.yaml

In [ ]:
kubectl apply -f acme_fitness_demo/kubernetes-manifests/acme_fitness_cluster2.yaml

[Acme Cluster 2 fabric overview](./imgs/11-acme-c2-fabric-overview.png)

## Create A Global Namespace

[Create a Global Namespace](./imgs/12-gns.png)

[General Details](./imgs/13-gns-general-details.png)

[Service Mappings](./imgs/14-gns-service-mapping.png)

[Public Services](./imgs/15-gns-public-services.png)

[External Services](./imgs/16-gns-external-services.png)

[Security](./imgs/17-gns-security.png)

[Summary](./imgs/18-gns-summary.png)

[Global Namespace homepage](./imgs/19-gns-homepage.png)

[Global Namespace fabric overview](./imgs/20-gns-fabric-overview.png)

## Remove Catalog Service On Cluster 1

In [ ]:
kubectl config use-context acme-c1

In [ ]:
kubectl set env deploy shopping CATALOG_HOST=catalog.acme.com

In [ ]:
kubectl delete -f acme_fitness_demo/kubernetes-manifests/acme_fitness_cluster2.yaml

In [ ]:
kubectl delete -f acme_fitness_demo/traffic-generator/loadgen.yaml

In [ ]:
kubectl apply -f acme_fitness_demo/traffic-generator/loadgen.yaml